In [1]:
import requests
import pandas as pd

In [8]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'

In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [9]:
driver = webdriver.Chrome('/Users/com/Downloads/chromedriver_win32/chromedriver')
driver.get(url)

In [16]:
divs = driver.find_elements_by_name('p')
divs


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=92.0.4515.159)


In [46]:
# soup.select('form > table > tbody > tr')      책에는 이렇게 나옴
channel_list = soup.select('.aos-init')
len(channel_list)

100

- 채널명

In [47]:
channel = channel_list[0]
category = channel.select_one('p.category').get_text().strip(' \n[]')
category

'음악/댄스/가수'

In [48]:
name = channel.select_one('.subject a').text.strip()
name

'BLACKPINK'

In [49]:
subscriber = channel.select_one('.subscriber_cnt').get_text()
view = channel.select_one('.view_cnt').text
video = channel.select_one('.video_cnt').string[:-1]
subscriber, view, video

('6400만', '190억0381만', '371')

In [50]:
#한방에
results = []
for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip(' \n[]')
    name = channel.select_one('.subject a').text.strip()
    subscriber = channel.select_one('.subscriber_cnt').get_text()
    view = channel.select_one('.view_cnt').text
    video = channel.select_one('.video_cnt').string[:-1]

    results.append([category, name, subscriber, view, video])

In [51]:
df = pd.DataFrame(channels, columns=['카테고리', '채널명', '구독자수', '조회수', '비디오수'])

df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,6400만,190억0381만,371
1,음악/댄스/가수,HYBE LABELS,6030만,187억1305만,654
2,음악/댄스/가수,BANGTANTV,5640만,121억9496만,"1,579"
3,음악/댄스/가수,SMTOWN,2850만,218억7092만,"3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223


- 만과 억을 숫자로 바꾸는 함수

In [57]:
def convert_unit(s):
    s = ''.join(s.split('억'))
    s = s.replace('억', '0000').replace('개', '').replace(',','')
    s = s.replace('만', '0000')
    return f'{int(s):,d}'

- xpath로 페이지 이동하기

In [55]:
# driver.find_element_by_xpath('//*[@id="list-skin"]/nav/span/a[1]').click()

## page1 에서 10까지 크롤링하기

In [67]:
results = []
for page in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    channel_list = soup.select('.aos-init')

    for channel in channel_list:
        category = channel.select_one('p.category').get_text().strip(' \n[]')
        name = channel.select_one('.subject a').text.strip()
        subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
        view = convert_unit(channel.select_one('.view_cnt').text)
        video = convert_unit(channel.select_one('.video_cnt').text)
        results.append([category,name,subscriber,view,video])

In [70]:
df = pd.DataFrame(results, columns=['카테고리', '채널명', '구독자수', '조회수', '비디오수'])

df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,"64,000,000","19,003,810,000",371
1,음악/댄스/가수,HYBE LABELS,"60,300,000","18,713,050,000",654
2,음악/댄스/가수,BANGTANTV,"56,400,000","12,194,960,000","1,579"
3,음악/댄스/가수,SMTOWN,"28,500,000","21,870,920,000","3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223


In [71]:
df.tail()

,카테고리,채널명,구독자수,조회수,비디오수
995,게임,미소,"510,000","210,770,000","3,084"
996,미분류,밉지않은 관종언니,"510,000","67,740,000",170
997,미분류,복지마블TV [Welfare Marble],"510,000","49,960,000",240
998,음식/요리/레시피,Muggo,"510,000","183,720,000","1,388"
999,게임,임선비,"510,000","234,290,000","1,134"


In [72]:
df.to_csv('유튜브_순위.csv', index=False)

In [73]:
driver.close()